#### Advection caclulations - version that calls functions from adfuncs file

In [2]:
#Import needed packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

from matplotlib import colors

# import cdflib
# from scipy.interpolate import RBFInterpolator

# #Use this to ignore warning in inerpolation script, maybe get rid of later to check for other issues
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', None) #Change settings in Pandas so it shows all columns

#Import functions from adfuncs
# import adfuncs

from adfuncs import hrz_ad_calc, prof_calcs, wind_prof

NameError: name 'pd' is not defined

In [2]:
#Constants
R = 8.314 #kgm^2s^-2K^-1mol^-1
h = 30 #Measurement height in m
L = 2.5*10**3 #Latent heat of vapoization of water at 0 deg C [J/g], should try to calc based on dependency on C
cp = 1005 #J/kg*K
mm = 0.02896 #kg/mol molar mass dry air
mm_w = 18.01 # molar mass of water [g/mol]
CtoK = 273.15 #Conversion from Celcius to Kelvin

In [3]:
#Read in all data

#Read in tower coords and set up dataframe
tc = pd.read_csv('../Inputs/tower_coords.csv') #datafrane of tower coordinates
tc.set_index('Tower', inplace = True)

#Convert lat and lon to coordinates in km, origin currently set to PFA coordinates, should change this since PFk is out of range
tc['x'] = (tc.Lon - tc.Lon['PFA'])*111*np.cos(tc.Lat['PFA']*np.pi/180)
tc['y'] = (tc.Lat - tc.Lat['PFA'])*111 


#Read in TA, H2O, and wind data
TA = pd.read_pickle('../Inputs/TA_orig_all.pickle')
H2O = pd.read_pickle('../Inputs/H2O_orig_all.pickle')
TA_grad = pd.read_pickle('../Inputs/TA_int_all.pickle')
H2O_grad = pd.read_pickle('../Inputs/H2O_int_all.pickle')
hrz_wind = pd.read_pickle('../Inputs/wind_dat.pickle')
vert_wind = pd.read_pickle('../Inputs/w_all.pickle')
PA_top = pd.read_pickle('../Inputs/PA_top.pickle')

#Rename vert_wind columns
tow_names =  dict(zip(tc.wind_var[1:], tc.index[1:]))
vert_wind = vert_wind.rename(columns = tow_names)

In [15]:
#Calc hrz adv for all 30 m towers (fiture out shorter towers later)
H_hrz_all = pd.DataFrame()
LE_hrz_all = pd.DataFrame()

for tower in tc.index:
    if tc.loc[tower].z1 == 30:
        H_hrz, LE_hrz = hrz_ad_calcl(tower)
        H_hrz_all[tower] = H_hrz
        LE_hrz_all[tower] = LE_hrz
        
hrz_ad = pd.concat([H_hrz_all, LE_hrz_all], keys = ['H', 'LE'])   

NameError: name 'tc' is not defined

In [12]:
def hrz_ad_calcl(tower, step = 0.1):
    

    WS_prof, T_grad_NS, T_grad_EW, H2O_grad_NS, H2O_grad_EW = prof_calcsl(tower, step)
    
    TA_K = TA.loc[30][tower] + CtoK
    
    PA_Pa = PA_top[tower]*1000 #kPa to Pa **this is at top of tower(~400m), need PA val for 30m and below**
    rho = mm*PA_Pa/TA_K/R #Dry air density at each tower [kg/m^3] **should probably use virtual temperature here** 
    z = step #np.asarray(tc.loc[tower].z1) #Height of each tower

    SN_wind = WS_prof.mul(np.sin(hrz_wind.loc['WD'][tower]*np.pi/180), axis = 0)
    WE_wind = WS_prof.mul(np.cos(hrz_wind.loc['WD'][tower]*np.pi/180), axis = 0)
    
    #H advection
    H_SN = T_grad_NS*SN_wind.mul(-rho*z*cp/2000, axis = 0) #Advection of sensible heat by southerly wind [W/m^2]
    H_WE = T_grad_EW*WE_wind.mul(-rho*z*cp/2000, axis = 0) #Advection of sensible heat by westerly wind [W/m^2]
    H_ad = H_SN + H_WE #Total advection

    #LE advection
    LE_SN = H2O_grad_NS*SN_wind.mul(-z*L/2000) #Advection of latend heat by southerly wind [W/m^2]
    LE_WE = H2O_grad_EW*WE_wind.mul(-z*L/2000 )#Advection of latent hear by westerly wind [W/m^2]
    LE_ad = (LE_SN + LE_WE)#*rho/rho #LE advection (the *rho/rho just makes it a df with the same tower and timestamp as indices)


    # hrz_ad = (H_ad + LE_ad)
    H_ad_tot = H_ad.sum(axis = 1)
    LE_ad_tot = LE_ad.sum(axis = 1)
    
    return H_ad_tot, LE_ad_tot
    #Deal with water vapor later
    # md_w = e/(R*TA_K) #Molar density of water vapor [mol/m^3], used in SLE calc
    

In [10]:
def prof_calcsl(tower, step = 0.1):
    
    global tc #Use tc defined in main notebook
    
    towdat = tc.loc[tower] #Tower data
    WS_prof = wind_prof(tower)
    
    h_tow = tc.z1.loc[tower] #tower height
    z_range = np.arange(0, h_tow + step, step)
    
    T_grad_NS, T_grad_EW = pd.DataFrame(), pd.DataFrame()
    H2O_grad_NS, H2O_grad_EW = pd.DataFrame(), pd.DataFrame()
    
   
    for z in z_range:     
        if towdat.z3 == 0:
            pass
        else:
            #should maybe make this more flexible for different tower heights
            T_grad_slope_NS_up = (TA_grad.loc['NS'].loc[30][tower] - TA_grad.loc['NS'].loc[10][tower])/20 #Slope of temp gradient from 10 to 30 m, NS
            T_grad_slope_EW_up = (TA_grad.loc['EW'].loc[30][tower] - TA_grad.loc['EW'].loc[10][tower])/20 #Slope of temp gradient from 10 to 30 m, EW
            T_grad_slope_NS_low = (TA_grad.loc['NS'].loc[10][tower] - TA_grad.loc['NS'].loc[2][tower])/8 #Slope of temp gradient from 2 to 10 m, NS
            T_grad_slope_EW_low = (TA_grad.loc['EW'].loc[10][tower] - TA_grad.loc['EW'].loc[2][tower])/8 #Slope of temp gradient from 2 to 10 m, EW
            
            H2O_grad_slope_NS_up = (H2O_grad.loc['NS'].loc[30][tower] - H2O_grad.loc['NS'].loc[10][tower])/20 #Slope of temp gradient from 10 to 30 m, NS
            H2O_grad_slope_EW_up = (H2O_grad.loc['EW'].loc[30][tower] - H2O_grad.loc['EW'].loc[10][tower])/20 #Slope of temp gradient from 10 to 30 m, EW
            H2O_grad_slope_NS_low = (H2O_grad.loc['NS'].loc[10][tower] - H2O_grad.loc['NS'].loc[2][tower])/8 #Slope of temp gradient from 2 to 10 m, NS
            H2O_grad_slope_EW_low = (H2O_grad.loc['EW'].loc[10][tower] - H2O_grad.loc['EW'].loc[2][tower])/8 #Slope of temp gradient from 2 to 10 m, EW
            
            if z >= towdat.z2:
                T_grad_NS[z] = TA_grad.loc['NS'].loc[10][tower] + T_grad_slope_NS_up*(z-towdat.z2)
                T_grad_EW[z] = TA_grad.loc['EW'].loc[10][tower] + T_grad_slope_EW_up*(z-towdat.z2)
                
                H2O_grad_NS[z] = H2O_grad.loc['NS'].loc[10][tower] + H2O_grad_slope_NS_up*(z-towdat.z2)
                H2O_grad_EW[z] = H2O_grad.loc['EW'].loc[10][tower] + H2O_grad_slope_EW_up*(z-towdat.z2)
            else:
                T_grad_NS[z] = TA_grad.loc['NS'].loc[10][tower] - T_grad_slope_NS_low*(towdat.z2 - z)
                T_grad_EW[z] = TA_grad.loc['EW'].loc[10][tower] - T_grad_slope_EW_low*(towdat.z2 - z)
                
                H2O_grad_NS[z] = H2O_grad.loc['NS'].loc[10][tower] - H2O_grad_slope_NS_low*(towdat.z2 - z)
                H2O_grad_EW[z] = H2O_grad.loc['EW'].loc[10][tower] - H2O_grad_slope_EW_low*(towdat.z2 - z)

                

            
    return WS_prof, T_grad_NS, T_grad_EW, H2O_grad_NS, H2O_grad_EW

